In [ ]:
import h5py
import numpy as np
import pandas as pd

from parse_pa_and_fly_script import process_fly




from flymaker_script import write_flyfile
from flymaker_script import get_param_list

from SIMION_run_full_microscope import run_microscope
from parse_results import move_to_h5
from parse_results import dataset_to_df

In [ ]:
fly_file = "C:\\Users\\carina\\SULI2024\\SIMION_files_2024\\full_microscope.fly2"
db_file = "C:\\Users\\carina\\SULI2024\\test\\SIMIONdb"

Running the simulations


In [ ]:
# Generate list of parameters: vary KE
ke_values = [300] #, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000]

tob = ['',0]
ke = ['gaussian', ke_values, 5]
azm = ['',0]
el = ['arithmetic sequence', 30, -2, 31]
direction = [azm , el]

# Setting some y spread.
source_y_pos = [-0.2, -0.1, 0, 0.1, 0.2]
source_position = ['single vector',20.001, source_y_pos, 0]

In [ ]:
# writes a fly file for every KE
for value in ke_values:
    ke[1] = value
    file_str = []
    for y in source_y_pos:
        source_position[2] = y
        
        file_str.append(get_param_list(tob, ke, direction, source_position))
          
    #print(file_str)

In [9]:
# Varying voltages
#v3_values = list(np.arange(start = 10000, step= 100, stop=26100)) # Includes start, does not include stop
v3_values = [14000,18000]
print(len(v3_values))
voltages = [0,2500,v3_values,2000,2500,5000,1100,338,2225,10,10,195,92,30,47]

2


In [17]:
for v3 in v3_values: # Each value of v3 is a new simulation. (Each set of voltages is a new simulation)
    voltages[2] = v3
    #print(voltages)
    
    for value in ke_values: # Each value of KE is a new simulation
        ke[1] = value
        file_str = []
      
        #print(ke)
        
        for y in source_y_pos: # Use a spread of y source coordinates
            source_position[2] = y
            file_str.append(get_param_list(tob, ke, direction, source_position))
            #print(get_param_list(tob, ke, direction, source_position))
        
        write_flyfile(file_name = fly_file, param_list = file_str)
        run_microscope(voltages)
        
        try:
            move_to_h5(h5 = db_file, fly_file = fly_file , v = voltages, ke_values = value, varying_electrode_id = 3)
            
        except Exception as inst:
            print(inst)

6
element: ['', 0]
element: 0.0005485799
element: -1
element: ['gaussian', 300, 5]
element: [['', 0], ['arithmetic sequence', 30, -2, 31]]
element: ['single vector', 20.001, 0.2, 0]
6
element: ['', 0]
element: 0.0005485799
element: -1
element: ['gaussian', 300, 5]
element: [['', 0], ['arithmetic sequence', 30, -2, 31]]
element: ['single vector', 20.001, 0.2, 0]


In [ ]:
# Delete existing datasets. Datasets cannot be overwritten or have duplicate names, execution order matters.
with h5py.File(db_file, "w") as f:
    for item in f.keys():
        del item
        
        for subitem in f[item].keys():
            del subitem

In [ ]:
# Check keys
with h5py.File(db_file, "r") as f:
    for item in f.keys():
        print(f'group: {item}')
    
        for subitem in f[item].keys():
            print(subitem)
    

In [ ]:
# Confirming the contents of the dataset:
with h5py.File(db_file, "r") as f:   
   
   # Retrieving the dataset
   dataset_name = 'v_0_2500_18000_2000_2500_5000_1100_338_2225_10_10_195_92_30_47_ke30_20240724_15020'
   sample_dataset = f['varying_v3'][dataset_name]
   
   # Check dataset attribute. Fly information is added as attributes to the corresponding dataset
   print(sample_dataset.attrs.keys()) 

   # Check dataset content. Returns one dataframe as a list of lists with the index matching the keys as they appear in run_results.
   #print(sample_dataset[0:6])
   
   # Dataset contents can be converted into a dataframe, but columns are unlabeled.
   sample_dataset_df = dataset_to_df(db_file,dataset_name)
   print(sample_dataset_df.head())
   
   

In [ ]:
voltages = [0,2500,14000,2000,2500,5000,1100,338,2225,10,10,195,92,30,47]
ke_values = [300] #, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000]

tob = ['',0]
ke = ['gaussian', ke_values, 5]
azm = ['',0]
el = ['arithmetic sequence', 30, -2, 31]
direction = [azm , el]

# Setting some y spread.
source_y_pos = [-0.2, -0.1, 0, 0.1, 0.2]
source_position = ['single vector',20.001, source_y_pos, 0]

write_flyfile(file_name = fly_file, param_list = file_str)
run_microscope(voltages)
        
try:
    move_to_h5(h5 = db_file, fly_file = fly_file , v = voltages, ke_values = 30, varying_electrode_id = 3)
            
except Exception as inst:
    print(inst)




In [ ]:
with h5py.File(db_file, 'r') as f:
    det_subset = []
    grp = f['varying_v3']
    #for item in grp:
        #print(item)
    dataset = grp['v_0_2500_10800_2000_2500_5000_1100_338_2225_10_10_195_92_30_47_ke300_20240724_150207']
    for items in dataset:
        if items[5] > 10000:
            det_subset.append(items[5])
    
    print(len(det_subset))

In [ ]:
import re

In [ ]:

def digit_check(str):

    if '-' in str:
        str = str.replace('-','')
        
    if '.' in str:
        str = str.replace('.','')
    
    if 'e' in str:
        str = str.replace('e','')

    return str.isdigit()

event_dict = {
    "Ion Created" : 1,
    "Crossed Y = 0 Plane" : 2,
    "Hit Electrode" : 3,
    "Outside Work Bench" : 4
    }

In [ ]:
def parse_results(results = "C:\\Users\\carina\\SULI2024\\SIMION_files_2024\\run_results.csv"):
    with open(results, 'r') as file:
        results_lines = file.readlines()
        
        flym_counter = 0
        start_parse = False
        data_dict = {}
        data_list = []
        datasets_list = []

        for row in results_lines:  
        
         if "Begin Fly'm" in row: # Begin Fly'm occurs for each simulation
            flym_counter +=1
                
         if 'Ion(' in row:
            start_parse = True
            if data_dict: # If the dictionary is not empty, dump it into the list
                data_list.append(data_dict)
                data_dict = {} # Reinitialize dictionary
         
         if start_parse:
            matches = re.findall('(.*?)\\((.*?)\\)', row) # Regex match
            key = []
            value = []
            for match in matches:
                key = match[0].strip()
                value = match[1].strip()
                
                #print(value)
                
                
                
                value_list = value.split(' ') # split value by space
                
               # print(value_list)
                
                if len(value_list) >= 2 and digit_check(value_list[0]):
                    value = float(value_list[0])
                    unit = value_list[1]
                    key = f'{key}_{unit}' 

                value = value if key != "Event" else event_dict[value] # change Event to numeric value
                data_dict[key] = float(value)
                
                #print(data_dict)
                
         else:
            start_parse = False 
         
        # In case multiple simulations are appended to one results file, a list of datasets are created
        if flym_counter >= 0:
            datasets_list.append(data_list)
            #data_list = [] / commented out on 7/24 to debug for the fact that we only see 5 particles at the detector
    print(data_list)
    
   
    # Will typically be a list of one (one simulation at a time)
    df_list = []
    for dataset in datasets_list:
        df = pd.DataFrame(dataset)
        df['Dataset_name'] = float(flym_counter)
        print(df.head())
        df_list.append(df)

    return df_list


In [ ]:
df_list = parse_results()
df = df_list[0]
det_data = df[df['X_mm'] > 10000]
det_data

Data processing

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
sns.set_theme('notebook')

In [ ]:
# Process all datasets to df
with h5py.File(db_file, 'r') as f:
    dataset_df_list = []
    
    # For every group,
    for item in f.keys():
        print(item)
        
        counter = -1
        # For every dataset in the group, convert dataset to a df and append it to a list
        for subitem in f[item]:
            counter += 1
            df = dataset_to_df(h5 = db_file, group_name = item, dataset_name = subitem) 
            df['v3_eV'] = v3_values[counter]
            dataset_df_list.append(df)
        
    print(len(dataset_df_list))
    #print(dataset_df_list[0])
    

In [ ]:
# Plot 1:  Y pos vs v3

# Merge datasets into one big df
all_v3_df = pd.concat(dataset_df_list)
all_v3_df[0:400]


In [ ]:
#pd.DataFrame({'Y_mm': [0,60]}).to_numpy()

pd.DataFrame({"Y_mm": [1, 2], "TOF_usec": [3, 4]}).to_numpy()

In [ ]:
y_arr[20000:21000]

In [ ]:
y_arr = all_v3_df['Y_mm'].to_numpy()
x_arr = all_v3_df['X_mm'].to_numpy()
x_filter = []
keep = False
for element in x_arr:
    if element > 9000:
        keep = True
    else:
        keep = False
    
    x_filter.append(keep)
        

print(len(x_filter))     

filter_count = []
for element in x_filter:
    if element == True:
        filter_count.append(element) 
        
print(len(filter_count))

"""
x_filters = [9999,15000]
filt_lon = np.logical_and.reduce([lon, filt])
filt_lon = np.logical_and.reduce([filt_lon, data>value[0], data<value[1]]) (e
"""

def filter_mask(arr, cond):
    return arr[cond(arr)]


#v3_arr = all_v3_df['v3_eV'].to_numpy() 
#plt.scatter(v3_arr, y_arr, marker ='.')

In [ ]:
sns.scatterplot(data = all_v3_df, x = 'v3_eV', y = 'Y_mm')
plt.xlim(17000,18000)


In [ ]:
detector_v3_df

In [ ]:
detector_v3_df = all_v3_df[all_v3_df['X_mm'] > 9000]
print(detector_v3_df.head())

sns.scatterplot(data = detector_v3_df, x = 'v3_eV', y = 'Y_mm', hue = 'v3_eV')
#plt.xticks(v3_values)
plt.xlim(17000,18000)
plt.ylim(-130,130)
plt.title('Detector data: Y position vs. Extractor voltage')

In [ ]:
y130df = all_v3_df[all_v3_df['Y_mm'] > 130]
y130df.describe()

In [ ]:
y_arr = detector_v3_df['Y_mm'].to_numpy()
v3_arr = detector_v3_df['v3_eV'].to_numpy() 
plt.scatter(v3_arr, y_arr, marker ='.', alpha = 0.2)
plt.xlim(10000,11000)

In [ ]:
# Plot 2: Collected particles vs. v3
sns.histplot (data = detector_v3_df, x = 'v3_eV', bins = 161, kde =True ) # change the bins
#plt.ylim(4.5, 5.05)
plt.title('Collected particles vs. Extractor Voltage')
plt.show()